In [0]:
#To read sample_sales.csv

file_schema =  """
id int,
name string,
dop string,
phone long,
amount string,
discount string
"""

In [0]:
sales_raw_df = (
    spark.read.format("csv")
    .option('header' , 'true')  
    .schema(file_schema)
    .load(path = '/Volumes/dev/spark_db/datasets/spark_programming/data/sales_sample.csv')
)
# if you are reading date format better dateFormat to tell the connector
sales_raw_df.display()
sales_raw_df.printSchema()

In [0]:
sales_raw_df.describe().display()

In [0]:
sales_raw_df.display()

In [0]:
from pyspark.sql.functions import expr,col
# Convert id from integer to string and rename it as transaction_id.
df  = sales_raw_df.withColumn('id' , expr("cast( id as string)"))
df = df.select(col('id').alias('transaction_id'),'name' , 'dop' , 'phone' , 'amount' , 'discount')
df.printSchema()
df.show()


In [0]:
sales_raw_df1 = sales_raw_df.selectExpr(
    "cast(id as string) as transaction_id" ,
    "name as customer_name" ,
    "nvl(try_cast(dop as date),to_date(dop , 'dd-MM-yyyy')) as date_of_purchase" ,
    "cast(phone as string) as customer_phone" ,
    "cast(amount as long) as purchase_amount" ,
#    "cast(case when discount is null then  '0' when discount = 'nil' then '0' else discount end as double) as discount" 
    "nvl(try_cast(discount as double),0) as applied_discount"
    )\
    .where("purchase_amount is not null and purchase_amount < 200000")


sales_raw_df1.printSchema()
sales_raw_df1.show()

In [0]:
sales_raw_df1.describe("purchase_amount" , "applied_discount").display()

Rename the name column to customer_name.

In [0]:
df = df.select('transaction_id',col('name').alias('customer_name') , 'dop' , 'phone' , 'amount' , 'discount')
df.printSchema()
df.show()

Convert the dop to date format and rename the column to date_of_purchase.

In [0]:
from pyspark.sql.functions import to_date

df = df.withColumn('date_of_purchase' , to_date(df.dop , 'dd/MM/yyyy'))
df = df.select('transaction_id','customer_name','date_of_purchase','phone','amount' , 'discount')
df.printSchema()
df.show()

Rename the phone column to customer_phone

In [0]:
from pyspark.sql.functions import col
df = df.select('transaction_id','customer_name','date_of_purchase',col('phone').alias('customer_phone'),'amount' , 'discount')
df.printSchema()


Convert the amount to a long value and filter out nulls and outlier values.

In [0]:
df.show()